# Importing Essential Modules and Packages

In [ ]:
import os
import pandas as pd
import librosa
import glob
import librosa.display
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

from sklearn import metrics
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from keras.callbacks import EarlyStopping

from keras import regularizers

from sklearn.preprocessing import LabelEncoder

from datetime import datetime


# Preparing the Data


In [ ]:
filelist = os.listdir("drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat")
cat_files = pd.DataFrame(filelist)
metadata = pd.read_csv("drive/MyDrive/ProjectMGN/CatandDog/train_test_split.csv")

In [ ]:
filelist = os.listdir("drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog")
dog_files = pd.DataFrame(filelist)

In [ ]:
cat_files.head()

In [ ]:
dog_files.head()

In [ ]:
metadata.head()

In [ ]:
cat_files

# Merge the Cat Dog Dataset and Shuffling the Dataset

In [ ]:
cat_files['label']='0'
dog_files['label']='1'
df = pd.concat([dog_files,cat_files],ignore_index=True)
df = df.rename(columns={0:'file'})
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
# Checking Class Balance

df['label'].value_counts()

In [ ]:
# Checking for Erroneous Folders

df[df['file']=='.DS_Store']

# Function to Extract Primary Features from Audio File

In [ ]:
def extract_features(data):
  # file_name - Directory to cat/dog audio files
  try:
    file_name = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat/'+str(data.file)
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  except FileNotFoundError:
    file_name = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog/'+str(data.file)
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  # print(file_name)
  mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
  stft = np.abs(librosa.stft(X))
  chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
  contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
  tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
  label = data.label
  return mfccs, chroma, mel, contrast, tonnetz, label


# Generating Features


In [ ]:
startTime=datetime.now()
features_labels = df.apply(extract_features, axis=1)
endtime=datetime.now()
print("Total Time Taken:",endtime-startTime)

In [ ]:
# Checking Feature Labels

features_labels

In [ ]:
print(f"len(feautures_labels):{len(features_labels)} and len(features_labels[0]):{len(features_labels[0])}")

# Saving the Feature Vector Generated

In [ ]:
np.save('drive/MyDrive/ProjectMGN/features_label_catdog_max_dataset',features_labels)

In [ ]:
features_labels=np.load('drive/MyDrive/ProjectMGN/features_label_catdog_max_dataset.npy', allow_pickle=True)

# Inputs for ANN

# Feature Vector

In [ ]:
features=[]
for i in range(0, len(features_labels)):
    features.append(np.concatenate((features_labels[i][0], features_labels[i][1], 
                features_labels[i][2], features_labels[i][3],
                features_labels[i][4]), axis=0))

In [ ]:
print(f"len(feautures):{len(features)} and len(features[0]):{len(features[0])}")

In [ ]:
X = np.array(features)
y = np.array(df['label'])

In [ ]:
# OneHotEncoding Labels

lb = LabelEncoder()
y = to_categorical(lb.fit_transform(y))

In [ ]:
X.shape

In [ ]:
y.shape

# Normalizing the Data

In [ ]:
ss = StandardScaler()
TEST_LIMIT = 250
x_train = X[:TEST_LIMIT]
y_train = y[:TEST_LIMIT]
x_test = X[TEST_LIMIT:]
y_test = y[TEST_LIMIT:]
X_train=ss.fit_transform(x_train)
X_test=ss.transform(x_test)

# Making and evaluating the ANN Model

In [ ]:
# Build a simple dense model with early stopping with softmax for categorical classification
# Total of 2 Classes - 0 representing Cat and representing  Dog

model = Sequential()

model.add(Dense(193, input_shape=(193,), activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='accuracy')

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

In [ ]:
history = model.fit(X_train, y_train, batch_size=256, epochs=100, validation_data=(X_test, y_test))

In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Set figure size.
plt.figure(figsize=(12, 8))

# Generate line plot of training, testing loss over epochs.
plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
plt.plot(val_accuracy, label='Validation Accuracy', color='orange')

# Set title
plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Categorical Crossentropy', fontsize = 18)
plt.xticks(range(0,100,5), range(0,100,5))
plt.legend(fontsize = 18);

In [ ]:
# Predicting On Test Data

predict_x=model.predict(X_test) 
preds=np.argmax(predict_x,axis=1)
preds = lb.inverse_transform(preds)
df_test = df[TEST_LIMIT:]
df_test['preds']=preds

In [ ]:
df_test[df_test['label'] != df_test['preds']]

In [ ]:
df_test[df_test['label'] != df_test['preds']]

In [ ]:
# Checking our model accuracy
1-round(len(df_test[df_test['label'] != df_test['preds']])/len(df_test),3)


# Loading and saving the model

In [ ]:
model.save('drive/MyDrive/Voice Detection/Saved Models/catdogmodel4error')

In [ ]:
import tensorflow as tf

new_model = tf.keras.models.load_model('drive/MyDrive/Voice Detection/Saved Models/catdogmodelmaxdb')

# Check its architecture
new_model.summary()

In [ ]:
# We get our predictions from the test data
predict_x=new_model.predict(X_test) 
preds=np.argmax(predict_x,axis=1)
preds = lb.inverse_transform(preds)
df_test = df[TEST_LIMIT:]
df_test['preds']=preds

# Generating Predict Function for WebApp

In [ ]:
def single_extract_features(data):
  try:
    file_name = str(data.file)
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  except FileNotFoundError:
    file_name = str(data.file)
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  # print(file_name)
  mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
  stft = np.abs(librosa.stft(X))
  chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
  contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
  tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
  label = data.label
  return mfccs, chroma, mel, contrast, tonnetz, label


def instant_predict(file):
  obj = pd.DataFrame({"file":file,"label":1},index=[0])
  features_labels = obj.apply(single_extract_features,axis=1)
  print(features_labels)
  features=[]
  for i in range(0, len(features_labels)):
      features.append(np.concatenate((features_labels[i][0], features_labels[i][1], 
                  features_labels[i][2], features_labels[i][3],
                  features_labels[i][4]), axis=0))
  X = np.array(features)
  #new_model = tf.keras.models.load_model('drive/MyDrive/Voice Detection/Saved Models/catdogmodel1')
  return model.predict(X)

In [ ]:
predict_x=instant_predict('drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/test/cats/cat_30.wav')
preds=np.argmax(predict_x,axis=1)
preds = lb.inverse_transform(preds)
print(preds)